In [1]:
import os

os.chdir("/home/hack12/ensemble-ai2024/")

In [2]:
import random
from pathlib import Path
from taskdataset import TaskDataset
import hydra
import numpy as np
import torch
import wandb
from omegaconf import omegaconf, OmegaConf
from torch import optim, nn
from torch.utils.data import DataLoader
from torchvision.transforms import RandomAffine
from tqdm import tqdm
import random
from src.config import Config
# from src.data.custom_dataset import EncodingsToLabels, EncodingsDataset
from src.models.linear_head import Net, MapperNet
from src.transforms.affine import AffineTransform, AffineAndPadAndShuffleTransform, PadAndShuffleTransform
from src.transforms.binary import BinaryTransform
import json
from typing import List
import requests
import argparse
from torch.utils.data import Dataset
# import end2end_stealing.vision_transformer as vits
import torchvision.models as models
import torchvision
import numpy as np

/home/hack12/ensemble-ai2024/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def train(epoch, model, train_loader, criterion, optimizer, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = criterion(output, target.to(device))
        loss.backward()
        optimizer.step()
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
               100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def get_wodzes(features, ids, n=10, near_dist=None):
    ids_to_return = []
    # features_to_return = []
    indexes = []

    random_index = random.randint(0, len(features))

    # features_to_return.append(features[random_index])
    ids_to_return.append(ids[random_index])
    indexes.append(random_index)

    # near_indices = []
    near_images = []

    for i in tqdm(range(n)):
        distances = torch.cdist(features[indexes], features)
        sum_distances = torch.sum(distances, dim=0, keepdim=True)
        sorted_indices = torch.argsort(sum_distances, descending=True)[0]
        # max_value_index = torch.argmax(sum_distances).item()
        # if near_dist is not None:
        #     max_val_distances = torch.cdist(features[max_value_index], features)
        #     mask = max_val_distances < near_dist
        #     indices = torch.nonzero(mask, as_tuple=False)
        #     near_indices.extend(indices)
        #     near_images.extend([dataset[i] for i in indices])
        for idx in sorted_indices:
            if idx not in indexes:
                max_value_index = idx.item()
                break

        indexes.append(max_value_index)
        ids_to_return.append(ids[max_value_index])

    # indexes.extend(near_indices)
    ids_to_return.extend(near_images)

    return ids_to_return

In [5]:
import json

In [33]:
def sybil_attack(ids: List[int], home_or_defense: str, binary_or_affine: str):
    if home_or_defense not in ["home", "defense"] or binary_or_affine not in ["binary", "affine"]:
        raise "Invalid endpoint"
    
    TEAM_TOKEN = "8J40ASDQOjfeeSKL"
    SERVER_URL = "http://34.71.138.79:9090"

    ENDPOINT = f"/sybil/{binary_or_affine}/{home_or_defense}"
    URL = SERVER_URL + ENDPOINT

    ids = ids = ",".join(map(str, ids))

    response = requests.get(
        URL, params={"ids": ids}, headers={"token": TEAM_TOKEN}
    )

    if response.status_code == 200:
        return json.loads(response.content)["representations"]
    else:
        raise Exception(f"Request failed. Status code: {response.status_code}, content: {response.content}")

In [6]:
def sybil_reset(binary_or_affine: str, home_or_defense: str):
    TEAM_TOKEN = "8J40ASDQOjfeeSKL"
    SERVER_URL = "http://34.71.138.79:9090"
    
    if binary_or_affine not in ["binary", "affine"]:
        raise Exception("Invalid endpoint")
    
    if home_or_defense not in ["home", "defense"]:
        raise Exception("Invalid endpoint")

    endpoint = f"/sybil/{binary_or_affine}/reset/{home_or_defense}"
    url = SERVER_URL + endpoint
    response = requests.post(url, headers={"token": TEAM_TOKEN})
    if response.status_code == 200:
        print("Request ok")
        print(response.json())
    else:
        raise Exception(
            f"Sybil reset failed. Code: {response.status_code}, content: {response.json()}"
        )

In [7]:
sybil_reset('affine', 'defense')

Request ok
{'msg': 'Successful sybil affine reset defense'}


In [37]:
class EncodingsDataset(Dataset):
    def __init__(self, transformed_encodings, encodings):
        self.transformed_encodings = torch.from_numpy(transformed_encodings.astype('float32'))
        self.encodings = torch.from_numpy(encodings.astype('float32'))

    def __len__(self):
        return len(self.transformed_encodings)

    def __getitem__(self, idx):
        return self.transformed_encodings[idx], self.encodings[idx]

In [9]:
SEED = 0

In [38]:

torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)
torch.backends.cudnn.benchmark = False
random.seed(SEED)
np.random.seed(SEED)

device = torch.device(f"cuda") if torch.cuda.is_available() else torch.device("cpu")

In [28]:
victim_model = models.__dict__["resnet50"]()
checkpoint = torch.load(
    "/home/hack12/ensemble-ai2024/end2end-stealing/pretrained_weights/checkpoint_0099.pth.tar",
    map_location="cpu",
)
state_dict = checkpoint["state_dict"]
for k in list(state_dict.keys()):
    # retain only encoder up to before the embedding layer
    if k.startswith("module.encoder") and not k.startswith("module.encoder.fc"):
        # remove prefix
        state_dict[k[len("module.encoder.") :]] = state_dict[k]
    # delete renamed or unused k
    del state_dict[k]
# print("state dict", state_dict.keys())
victim_model.load_state_dict(state_dict, strict=False)
victim_model.fc = torch.nn.Identity()
victim_model.cuda()
victim_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
dataset = torch.load('/home/hack12/ensemble-ai2024/src/data/SybilAttack.pt')

In [29]:
features = []
to_tensor = torchvision.transforms.ToTensor()

for iid, img, label in dataset:
    with torch.no_grad():
        img = img.convert("RGB")
        img = to_tensor(img).unsqueeze(0)

        victim_model(img.to("cuda"))
        features.append(victim_model(img.to("cuda")).cpu().numpy())

In [34]:
features = torch.from_numpy(np.array(features))
features = features.view(features.shape[0], -1)

In [39]:
TRANSFORMATION = 'affine'
OVERLAP = 1000
learning_rate = 3e-5
batch_size = 64
mapper_epochs = 100

In [40]:
transformation = TRANSFORMATION
overlap = OVERLAP

In [41]:
results = {}
EVAL_B_SIZE = 2000 - overlap

In [42]:
dataset_ids = dataset.ids

# np.save('ids_A', ids_A)
ids_A = np.load('ids_A.npy')
# ids_A = np.array(random.sample(dataset_ids, 2000))
# ids_A = get_wodzes(features, dataset_ids, n=2000)
unused_ids = [id for id in dataset_ids if id not in ids_A]


In [43]:
overlapping_examples_index = random.sample(range(0, 2000), overlap)
overlapping_examples = ids_A[overlapping_examples_index]

n_items = overlap

In [44]:
# Pobieram te dane z A, 2 tysiące


# real_data_A = sybil_attack(ids_A.tolist(), 'home', transformation)
# real_data_A = np.array(real_data_A)

# np.save('real_A_data', real_data_A)
real_data_A = np.load('real_A_data.npy')

In [45]:
# te dane z 2000 wybrane do treningu 
train_embeds_A = real_data_A[overlapping_examples_index]

results['ids'] = ids_A
results['features'] = real_data_A

In [46]:
len(results['ids']), len(results['features'])

(2000, 2000)

In [47]:
MODEL_OUTPUT = real_data_A[0].shape[0]

In [21]:
# train_embeds_B = sybil_attack(overlapping_examples, 'defense', transformation)
# train_embeds_B = np.array(train_embeds_B)

In [22]:

# if len(unused_ids) >= EVAL_B_SIZE:
#     eval_B_index = random.sample(range(0, len(unused_ids)), EVAL_B_SIZE)
#     eval_B_ids = np.array(unused_ids)[eval_B_index]
#     unused_ids = [id for id in unused_ids if id not in eval_B_ids]
# else:
#     eval_B_ids = unused_ids
#     unused_ids = []

In [23]:
# eval_embeds_B = sybil_attack(eval_B_ids, 'defense', transformation)
# eval_embeds_B = torch.from_numpy(np.array(eval_embeds_B).astype('float32'))

In [24]:
# train_dataset = EncodingsDataset(train_embeds_B, train_embeds_A)

# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [25]:
# mapper = MapperNet(train_embeds_B[0].shape[0],
#                     train_embeds_A[0].shape[0],
#                     n_hidden=MODEL_OUTPUT).to(device)
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(mapper.parameters(), lr=learning_rate)

# print("Training mapper...")
# for epoch in range(1, mapper_epochs):
#     train(epoch, mapper, train_loader, criterion, optimizer, device=device)

In [48]:
import time

In [49]:
while unused_ids:

    sybil_reset('affine', 'defense')

    time.sleep(20)

    # print('overlapping len', len(overlapping_examples))

    # train_embeds_B = sybil_attack(overlapping_examples, 'defense', transformation)
    # train_embeds_B = np.array(train_embeds_B)


    if len(unused_ids) >= EVAL_B_SIZE:
        eval_B_index = random.sample(range(0, len(unused_ids)), EVAL_B_SIZE)
        eval_B_ids = np.array(unused_ids)[eval_B_index]
        unused_ids = [id for id in unused_ids if id not in eval_B_ids]
    else:
        eval_B_ids = unused_ids
        unused_ids = []

    print('unused_ids', len(unused_ids))
    
    ask_for_B = np.concatenate((overlapping_examples, eval_B_ids))

    # eval_embeds_B = sybil_attack(eval_B_ids, 'defense', transformation)
    # eval_embeds_B = torch.from_numpy(np.array(eval_embeds_B).astype('float32'))

    print('ask len', len(ask_for_B))
    
    for i in range(20):
        try:
            asked_embeds_B = sybil_attack(ask_for_B, 'defense', transformation)
            break
        except:
            sybil_reset('affine', 'defense')
            time.sleep(2)
            continue
    
    train_embeds_B = asked_embeds_B[:len(overlapping_examples)]
    train_embeds_B = np.array(train_embeds_B)
    eval_embeds_B = asked_embeds_B[len(overlapping_examples):]
    eval_embeds_B = torch.from_numpy(np.array(eval_embeds_B).astype('float32'))

    train_dataset = EncodingsDataset(train_embeds_B, train_embeds_A)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    mapper = MapperNet(train_embeds_B[0].shape[0],
                        train_embeds_A[0].shape[0],
                        n_hidden=MODEL_OUTPUT).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(mapper.parameters(), lr=learning_rate)

    print("Training mapper...")
    for epoch in range(1, mapper_epochs):
        train(epoch, mapper, train_loader, criterion, optimizer, device=device)

    reconstructed_encodings_test = []
    for encoding in tqdm(eval_embeds_B):
        reconstructed_encoding = mapper(encoding.to(device))
        reconstructed_encodings_test.append(reconstructed_encoding.detach().to("cpu"))
    reconstructed_encodings_test = torch.cat(reconstructed_encodings_test).reshape(
        len(eval_embeds_B), -1)

    results['ids'] = np.concatenate((results['ids'], eval_B_ids))
    results['features'] = np.concatenate((results['features'], reconstructed_encodings_test.numpy()))

    print('results len', len(results['ids']))

    # sybil_reset('affine', 'defense')

Request ok
{'msg': 'Successful sybil affine reset defense'}


In [35]:
len(results['ids'])

18500

In [134]:
np.savez(
        "example_submission.npz",
        ids=results['ids'],
        representations=results['features'],
    )

In [ ]:
# Train a mapper
while unused_ids:
    
    train_embeds_B = sybil_attack(overlapping_examples, 'defense', transformation)
    
    if len(unused_ids) >= EVAL_B_SIZE:
        eval_B_ids = random.sample(range(0, len(unused_ids)), EVAL_B_SIZE)
        unused_ids = [id for id in unused_ids if id not in eval_B_ids]
    else:
        eval_B_ids = unused_ids
        unused_ids = []

    eval_embeds_B = sybil_attack(eval_B_ids, 'defense', transformation)

    train_dataset = EncodingsDataset(train_embeds_B, train_embeds_A)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    mapper = MapperNet(train_embeds_B[0][0].shape[0],
                        train_embeds_A[0][0].shape[0],
                        n_hidden=MODEL_OUTPUT).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(mapper.parameters(), lr=learning_rate)

    print("Training mapper...")
    for epoch in range(1, mapper_epochs):
        train(epoch, mapper, train_loader, criterion, optimizer, device=device)

    reconstructed_encodings_test = []
    for encoding in tqdm(eval_embeds_B):
        reconstructed_encoding = mapper(encoding.to(device))
        reconstructed_encodings_test.append(reconstructed_encoding.detach().to("cpu"))
    reconstructed_encodings_test = torch.cat(reconstructed_encodings_test).reshape(
        len(eval_embeds_B), -1)

    results['ids'] = np.concatenate(results['ids'], eval_B_ids)
    results['features'] = np.concatenate(results['features'], reconstructed_encoding)

In [ ]:
get_results(TRANSFORMATION, OVERLAP)